In [1]:
pip install arxiv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install ollama


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install anthropic

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import arxiv
import json
import os
from typing import List, Dict
from dotenv import load_dotenv
import ollama


In [5]:
PAPER_DIR = "papers"

In [6]:
def search_papers(topic: str, max_results: int = 5) -> List[str]:
    """Search for papers on arXiv related to the given topic."""

    client = arxiv.Client()

    # Search for most relevant articles matching the query topics.
    search = arxiv.Search(
        query=topic,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.Relevance
    )
    papers = client.results(search)

    # Creating directory of the topic.
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)

    file_path = os.path.join(path, "papers.json")

    # Try to load existing paper info.
    papers_info = {}
    paper_ids = []
    for paper in papers:
        paper_ids.append(paper.get_short_id())
        paper_info = {
            'title': paper.title,
            'authors': [author.name for author in paper.authors],
            'summary': paper.summary,
            'pdf_url': paper.pdf_url,
            'published': str(paper.published.date())
        }
        papers_info[paper.get_short_id()] = paper_info

    # Save updated papers into JSON file.
    with open(file_path, 'w') as json_file:
        json.dump(papers_info, json_file, indent=2)
    print(f"Results are saved to {file_path}")
    return paper_ids


In [7]:
search_papers("machine learning")

Results are saved to papers\machine_learning\papers.json


['2306.04338v1',
 '2006.16189v4',
 '2201.12150v2',
 '2302.08893v4',
 '2304.02381v2']

In [8]:
def extract_info(paper_id: str) -> str:
    """Extract title and abstract from a paper given its arXiv ID."""
    for item in os.listdir(PAPER_DIR):
        item_path = os.path.join(PAPER_DIR, item)
        if os.path.isdir(item_path):
            file_path = os.path.join(item_path, "papers.json")
            if os.path.exists(file_path):
                try:
                    with open(file_path, 'r') as json_file:
                        papers_info = json.load(json_file)
                        if paper_id in papers_info:
                            return json.dumps(papers_info[paper_id], indent=2)
                except (FileNotFoundError, json.JSONDecodeError) as e:
                    print(f"Error reading {file_path}: {str(e)}")
                    continue

    return f"There is no information found for paper ID {paper_id}."


In [9]:
extract_info("2201.12150v2")

'{\n  "title": "Learning Curves for Decision Making in Supervised Machine Learning: A Survey",\n  "authors": [\n    "Felix Mohr",\n    "Jan N. van Rijn"\n  ],\n  "summary": "Learning curves are a concept from social sciences that has been adopted in the context of machine learning to assess the performance of a learning algorithm with respect to a certain resource, e.g., the number of training examples or the number of training iterations. Learning curves have important applications in several machine learning contexts, most notably in data acquisition, early stopping of model training, and model selection. For instance, learning curves can be used to model the performance of the combination of an algorithm and its hyperparameter configuration, providing insights into their potential suitability at an early stage and often expediting the algorithm selection process. Various learning curve models have been proposed to use learning curves for decision making. Some of these models answer 

## Tool Mapping

In [10]:
mapping_tool_function = {
    "search_papers": search_papers,
    "extract_info": extract_info
}

def execute_tool(tool_name, tool_args):
    
    result = mapping_tool_function[tool_name](**tool_args)

    if result is None:
        result = "The operation completed but didn't return any results."
        
    elif isinstance(result, list):
        result = ', '.join(result)
        
    elif isinstance(result, dict):
        # Convert dictionaries to formatted JSON strings
        result = json.dumps(result, indent=2)
    
    else:
        # For any other type, convert using str()
        result = str(result)
    return result

In [11]:
tools = [
    {
        "name": "search_papers",
        "description": "Search for papers on arXiv related to a given topic.",
        "input_schema": {
            "type": "object",
            "properties": {
                "topic": {
                    "type": "string",
                    "description": "The topic to search for papers about"
                },
                "max_results": {
                    "type": "integer",
                    "description": "Maximum number of papers to return (default: 5)"
                }
            },
            "required": ["topic"]
        }
    },
    {
        "name": "extract_info",
        "description": "Extract detailed information (title, authors, summary, etc.) from a paper given its arXiv ID.",
        "input_schema": {
            "type": "object",
            "properties": {
                "paper_id": {
                    "type": "string",
                    "description": "The arXiv ID of the paper (e.g., '2201.12150v2')"
                }
            },
            "required": ["paper_id"]
        }
    }
]


In [12]:
import os

# Ollama setup - runs locally, no API key needed!
# Make sure Ollama is running in the background (ollama serve)
# And you've pulled a model: ollama pull mistral

OLLAMA_MODEL = 'mistral'  # Change to 'neural-chat' or 'llama2' if you prefer


In [13]:
def process_query(query):
    """
    Process user query with tool use capability.
    The AI can decide to call search_papers or extract_info tools.
    """
    try:
        if not query or query.strip() == "":
            print("Please enter a valid query")
            return
        
        # Build system prompt that tells Ollama about available tools
        system_prompt = """You are an AI assistant specialized in searching and analyzing academic papers on arXiv.

You have access to the following tools:
1. search_papers(topic: str, max_results: int = 5) - Search for papers on arXiv by topic
2. extract_info(paper_id: str) - Get detailed info about a specific paper

When the user asks you to search for papers, use the search_papers tool.
When the user asks about a specific paper, use the extract_info tool.

If you decide to use a tool, respond in this exact format:
TOOL_CALL: [tool_name] | {json_with_args}

For example:
TOOL_CALL: search_papers | {"topic": "machine learning", "max_results": 5}

After getting tool results, provide a helpful response to the user."""

        # First, ask Ollama what it thinks it should do
        full_prompt = f"{system_prompt}\n\nUser: {query}"
        
        response = ollama.generate(
            model=OLLAMA_MODEL, 
            prompt=full_prompt,
            stream=False
        )
        
        if not response or 'response' not in response:
            print("Could not generate a response. Try a different query.")
            return
        
        assistant_response = response['response'].strip()
        
        # Check if the assistant wants to call a tool
        if "TOOL_CALL:" in assistant_response:
            # Extract the tool call
            tool_call_idx = assistant_response.find("TOOL_CALL:")
            tool_section = assistant_response[tool_call_idx:].split('\n')[0]
            
            try:
                # Parse: TOOL_CALL: search_papers | {"topic": "algebra"}
                parts = tool_section.replace("TOOL_CALL:", "").strip().split("|")
                if len(parts) == 2:
                    tool_name = parts[0].strip()
                    tool_args = json.loads(parts[1].strip())
                    
                    print(f"\n🔍 Using tool: {tool_name}")
                    print(f"   Args: {tool_args}\n")
                    
                    # Execute the tool
                    tool_result = execute_tool(tool_name, tool_args)
                    
                    print(f"📄 Tool result:\n{tool_result}\n")
                    
                    # Now ask Ollama to provide a helpful response based on the tool result
                    followup_prompt = f"{system_prompt}\n\nUser: {query}\n\nTool used: {tool_name}\nTool result:\n{tool_result}\n\nProvide a helpful summary for the user:"
                    
                    followup_response = ollama.generate(
                        model=OLLAMA_MODEL,
                        prompt=followup_prompt,
                        stream=False
                    )
                    
                    if followup_response and 'response' in followup_response:
                        print(f"💬 Assistant: {followup_response['response'].strip()}")
                    return
            except (json.JSONDecodeError, ValueError) as e:
                print(f"Could not parse tool call: {e}")
                print(f"Response: {assistant_response}")
                return
        
        # No tool call, just print the response
        print(f"💬 Assistant: {assistant_response}")
            
    except ConnectionError:
        print("❌ Error: Ollama is not running!")
        print("Start Ollama with: & \"C:\\Users\\geek9\\AppData\\Local\\Programs\\Ollama\\ollama.exe\" serve")
    except Exception as e:
        print(f"❌ Error: {str(e)}")

In [14]:
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("\nQuery: ").strip()
            if query.lower() == 'quit':
                break
    
            process_query(query)
            print("\n")
        except Exception as e:
            print(f"\nError: {str(e)}")

In [15]:
print("=" * 60)
print("🤖 ArXiv Paper Search Assistant (Powered by Ollama + Mistral)")
print("=" * 60)
print("\n✨ I can help you search academic papers on arXiv!")
print("\nExamples of what you can ask:")
print("  • 'Search for papers on machine learning'")
print("  • 'Find papers about deep learning'")
print("  • 'What papers exist on quantum computing?'")
print("  • 'Tell me about paper 2201.12150v2'")
print("\nType 'quit' to exit.\n")
print("=" * 60)

🤖 ArXiv Paper Search Assistant (Powered by Ollama + Mistral)

✨ I can help you search academic papers on arXiv!

Examples of what you can ask:
  • 'Search for papers on machine learning'
  • 'Find papers about deep learning'
  • 'What papers exist on quantum computing?'
  • 'Tell me about paper 2201.12150v2'

Type 'quit' to exit.



In [ ]:
chat_loop()

Type your queries or 'quit' to exit.

🔍 Using tool: search_papers
   Args: {'topic': 'Machine Learning', 'max_results': 10}


🔍 Using tool: search_papers
   Args: {'topic': 'Machine Learning', 'max_results': 10}

Results are saved to papers\machine_learning\papers.json
📄 Tool result:
2306.04338v1, 2006.16189v4, 2201.12150v2, 2302.08893v4, 2304.02381v2, 2303.15563v1, 1905.04749v2, 1705.05172v1, 1906.01101v1, 2404.12511v1

Results are saved to papers\machine_learning\papers.json
📄 Tool result:
2306.04338v1, 2006.16189v4, 2201.12150v2, 2302.08893v4, 2304.02381v2, 2303.15563v1, 1905.04749v2, 1705.05172v1, 1906.01101v1, 2404.12511v1

💬 Assistant: Here are some interesting papers I found on arXiv related to your query. They cover topics such as machine learning, physics, and mathematics.

1. "A Fast Learning Algorithm for Sparse Linear Models" (2306.04338v1)
2. "Review of Stochastic Optimization Algorithms" (2006.16189v4)
3. "Statistical Mechanics of Machine Learning" (2201.12150v2)
4. "On t